## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.80,-68.30,48.20,87,75,5.82,broken clouds
1,1,Qaanaaq,GL,77.48,-69.36,5.50,82,0,4.90,clear sky
2,2,Busselton,AU,-33.65,115.33,57.00,93,100,1.99,overcast clouds
3,3,Kollam,IN,8.88,76.60,78.80,100,20,2.51,mist
4,4,Hithadhoo,MV,-0.60,73.08,83.89,70,100,8.77,light rain


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Kollam,IN,8.88,76.60,78.80,100,20,2.51,mist
4,4,Hithadhoo,MV,-0.60,73.08,83.89,70,100,8.77,light rain
6,6,Rikitea,PF,-23.12,-134.97,76.14,67,0,8.08,clear sky
7,7,Boundiali,CI,9.52,-6.49,72.86,55,2,0.67,clear sky
8,8,Aguimes,ES,27.91,-15.45,78.80,24,0,12.75,clear sky
9,9,Extremoz,BR,-5.71,-35.31,78.80,88,20,4.70,few clouds
10,10,East London,ZA,-33.02,27.91,65.01,74,97,12.19,overcast clouds
11,11,Vila Velha,BR,-20.33,-40.29,71.60,83,90,10.29,light rain
14,14,Hilo,US,19.73,-155.09,84.20,62,90,9.17,overcast clouds
16,16,Saint George,US,37.10,-113.58,69.80,15,1,3.36,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                291
City                   291
Country                291
Lat                    291
Lng                    291
Max Temp               291
Humidity               291
Cloudiness             291
Wind Speed             291
Current Description    291
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Kollam,IN,8.88,76.60,78.80,100,20,2.51,mist
4,4,Hithadhoo,MV,-0.60,73.08,83.89,70,100,8.77,light rain
6,6,Rikitea,PF,-23.12,-134.97,76.14,67,0,8.08,clear sky
7,7,Boundiali,CI,9.52,-6.49,72.86,55,2,0.67,clear sky
8,8,Aguimes,ES,27.91,-15.45,78.80,24,0,12.75,clear sky
...,...,...,...,...,...,...,...,...,...,...
652,652,Rafai,CF,4.95,23.92,69.60,97,100,1.63,light rain
653,653,Prado,BR,-17.34,-39.22,72.88,92,100,8.61,heavy intensity rain
654,654,Sampit,ID,-2.53,112.95,88.25,62,31,2.35,scattered clouds
657,657,Natal,BR,-5.79,-35.21,78.80,83,20,5.82,few clouds


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kollam,IN,78.80,mist,8.88,76.60,
4,Hithadhoo,MV,83.89,light rain,-0.60,73.08,
6,Rikitea,PF,76.14,clear sky,-23.12,-134.97,
7,Boundiali,CI,72.86,clear sky,9.52,-6.49,
8,Aguimes,ES,78.80,clear sky,27.91,-15.45,
9,Extremoz,BR,78.80,few clouds,-5.71,-35.31,
10,East London,ZA,65.01,overcast clouds,-33.02,27.91,
11,Vila Velha,BR,71.60,light rain,-20.33,-40.29,
14,Hilo,US,84.20,overcast clouds,19.73,-155.09,
16,Saint George,US,69.80,clear sky,37.10,-113.58,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df.head(10)
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kollam,IN,78.80,mist,8.88,76.60,The Raviz Ashtamudi
4,Hithadhoo,MV,83.89,light rain,-0.60,73.08,Scoop Guest House
6,Rikitea,PF,76.14,clear sky,-23.12,-134.97,Pension Maro'i
7,Boundiali,CI,72.86,clear sky,9.52,-6.49,Hotel Paysan Boundiali
8,Aguimes,ES,78.80,clear sky,27.91,-15.45,Hotel Villa de Aguimes
9,Extremoz,BR,78.80,few clouds,-5.71,-35.31,Casa do Mochileiro Airport Hostel
10,East London,ZA,65.01,overcast clouds,-33.02,27.91,Tu Casa
11,Vila Velha,BR,71.60,light rain,-20.33,-40.29,Hotel Vitória Palace
14,Hilo,US,84.20,overcast clouds,19.73,-155.09,Hilo Hawaiian Hotel
16,Saint George,US,69.80,clear sky,37.10,-113.58,Best Western Plus Abbey Inn


In [13]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   291
Country                291
Max Temp               291
Current Description    291
Lat                    291
Lng                    291
Hotel Name             291
dtype: int64

In [16]:
hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kollam,IN,78.80,mist,8.88,76.60,The Raviz Ashtamudi
4,Hithadhoo,MV,83.89,light rain,-0.60,73.08,Scoop Guest House
6,Rikitea,PF,76.14,clear sky,-23.12,-134.97,Pension Maro'i
7,Boundiali,CI,72.86,clear sky,9.52,-6.49,Hotel Paysan Boundiali
8,Aguimes,ES,78.80,clear sky,27.91,-15.45,Hotel Villa de Aguimes
...,...,...,...,...,...,...,...
652,Rafai,CF,69.60,light rain,4.95,23.92,
653,Prado,BR,72.88,heavy intensity rain,-17.34,-39.22,INN CORNER RIVER - PRADO - BAHIA - HOUSING AND...
654,Sampit,ID,88.25,scattered clouds,-2.53,112.95,Aquarius Boutique Hotel Sampit
657,Natal,BR,78.80,few clouds,-5.79,-35.21,Villa Park Hotel | Natal


In [18]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))